In [ ]:
!pip install pandas
!pip install emoji
!pip install --upgrade openai
!pip install ekphrasis -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai
from google.colab import files
import os
import pandas as pd
import time

In [ ]:
openai.api_key = "Input_your_api_key"
def translate_to_arabic(text):
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant that translates English to Arabic"},
            {"role": "user", "content": text}
        ],
      temperature=0.1,
      top_p = 0.9 # Set top_p parameter
    )
    
    translated_text = response['choices'][0]['message']['content']
    return translated_text

In [ ]:
# Upload the train, test, and dev txt files
uploaded = files.upload()

Saving 2018-E-c-En-dev.txt to 2018-E-c-En-dev.txt
Saving 2018-E-c-En-test-gold.txt to 2018-E-c-En-test-gold.txt
Saving 2018-E-c-En-train.txt to 2018-E-c-En-train.txt


In [ ]:
# Read train, test, and dev datasets into pandas DataFrames (Read for google collab)
train_df = pd.read_csv('2018-E-c-En-train.txt', delimiter='\t') # Update delimiter if necessary
test_df = pd.read_csv('2018-E-c-En-test-gold.txt', delimiter='\t') # Update delimiter if necessary
dev_df = pd.read_csv('2018-E-c-En-dev.txt', delimiter='\t') # Update delimiter if necessary


# Concatenate the DataFrames
df = pd.concat([train_df, test_df, dev_df], ignore_index=True)
df.rename(columns={"Tweet" : "content"}, inplace=True)

In [ ]:
# Replace Emoji, Remove (hashtag, punctuation, url, extra_whitespace, translate to lowercase)
import re
import string
import nltk
from nltk.tokenize import word_tokenize
import emoji  # We'll use this library to translate emojis

nltk.download('punkt')

def translate_emojis(text):
    return emoji.demojize(text)

def remove_hashtags(text):
    return re.sub(r'#\w+', '', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)

def to_lowercase(text):
    return text.lower()

def remove_extra_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

def Num(text):
    return re.sub(r'[0-9]', '', text)

def preprocess_text(text):
    #text = remove_urls(text)
    text = remove_mentions(text)
    #text = remove_hashtags(text)
    text = translate_emojis(text)  # Use the function to translate emojis
    #text = remove_html_tags(text)
    text = remove_punctuation(text)
    #text = to_lowercase(text)
    #text = remove_extra_whitespace(text)
    #text = Num(text)
    return text

def preprocess_dataframes(combined_data):
    combined_data['content'] = combined_data['content'].apply(preprocess_text)
    # This was the issue: no return statement
    return combined_data
    

# Assuming 'combined_data' is already DataFrames
df_pr = preprocess_dataframes(df)

# Save the preprocessed DataFrames to CSV files
#df.to_csv('preprocessed_all_data.csv', index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Apply max_match to seperate joint words
import nltk
from nltk.corpus import words

nltk.download('words')  # Download the 'words' corpus if it's not already downloaded

english_vocab = set(words.words())  # Use NLTK's set of English words

def max_match(word):
    if not word:
        return []
    for i in range(len(word), -1, -1):  # Start at the end of the word
        first = word[:i]
        rem = word[i:]
        if first.lower() in english_vocab:
            return [first] + max_match(rem)
    # If no word could be formed, make a single-character word and move on
    first = word[:1]
    rem = word[1:]
    return [first] + max_match(rem)

# Count words longer than 15 characters
long_words = df['content'].apply(lambda tweet: [word for word in tweet.split() if len(word) > 15])
long_words_count = sum(long_words.apply(len))

print(f"Number of words longer than 15 characters: {long_words_count}")

import pandas as pd

# Find all words longer than 15 characters and their splits
long_words_data = []
df['content'].apply(lambda tweet: [long_words_data.append((word, max_match(word))) for word in tweet.split() if len(word) > 15])

# Create DataFrame
long_words_df = pd.DataFrame(long_words_data, columns=['Original', 'Split'])

# Display DataFrame
print(long_words_df)

# Separate long words
df['content'] = df['content'].apply(lambda tweet: ' '.join(word for word_part in (max_match(word) if len(word) > 15 else [word] for word in tweet.split()) for word in word_part))


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Number of words longer than 15 characters: 2202
                         Original  \
0                decentishtonight   
1              Kaitupsidedownface   
2          shouldofgonewithvirgin   
3              enragedfacecatface   
4              facewithtearsofjoy   
...                           ...   
2197     smilingfacewithhearteyes   
2198  grinningfacewithsmilingeyes   
2199            taylorrayholbrook   
2200   someonegetmeajerseyteeasap   
2201             ThingsIveLearned   

                                             Split  
0                         [decent, is, h, tonight]  
1                     [Kai, tup, sided, own, face]  
2                 [should, of, gone, with, virgin]  
3                         [enraged, face, catface]  
4                   [face, with, tear, so, f, joy]  
...                                            ...  
2197          [smiling, face, with, heart, eye, s]  
2198       [grinning, face, with, smiling, eye, s]  
2199               [tay, lo, r

In [ ]:
'''from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    #tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

df['content'] = df['content'].apply(text_processor.pre_process_doc)

# Save the DataFrame as an Excel file
df[['content', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].to_excel('ekpharsis_data.xlsx', index=False)

# Download the Excel file
files.download('ekpharsis_data.xlsx')

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
or_df = df[0:20].copy()

In [ ]:
or_df

,ID,content,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2017-En-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,2017-En-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,2017-En-21068,it also helps that the majority of NFL coachin...,1,0,1,0,1,0,1,0,0,0,0
3,2017-En-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,2017-En-22195,My roommate its okay that we cant spell becaus...,1,0,1,0,0,0,0,0,0,0,0
5,2017-En-22190,No but thats so cute Atsu was probably shy abo...,0,0,0,0,1,0,0,0,0,0,0
6,2017-En-20221,Do you think humans have the sense for recogni...,0,1,0,0,0,0,0,1,0,0,0
7,2017-En-22180,Rooneys fucking untouchable isnt he Been fucki...,1,0,1,0,0,0,0,0,0,0,0
8,2017-En-41344,its pretty depressing when u hit pan on ur fav...,0,0,1,0,0,0,0,0,1,0,0
9,2017-En-20759,but your pussy was weak from what I heard so s...,1,0,1,0,0,0,0,0,0,0,0


In [ ]:
'''# Function to check if text contains any Arabic characters
def contains_arabic(text):
    return bool(re.search('[\u0600-\u06FF]', text))


# Create a new column 'arabic_text' by applying the translation function to 'content'
# Also, add a delay to adhere to the rate limit (3 requests per minute)
for i, row in or_df.iterrows():
    translated_text = ""
    attempts = 0  # counter for attempts
    
    while not contains_arabic(translated_text) and attempts < 2:
        translated_text = translate_to_arabic(row['content'])
        time.sleep(20)  # Pause execution for 20 seconds (60 seconds / 3 requests)
        attempts += 1  # increase the counter each time the loop runs

    or_df.at[i, 'arabic_text'] = translated_text

print(or_df)

In [ ]:
# Delay to adhere to the rate limit (3 requests per minute)
for i, row in or_df.iterrows():
    or_df.at[i, 'arabic_text'] = translate_to_arabic(row['content'])
    time.sleep(20)  # Pause execution for 20 seconds (60 seconds / 3 requests)

print(or_df)

In [ ]:
or_df[['content', 'arabic_text', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].to_csv('translated_data(temperature = 0.1 & top_p = 0.9).txt', sep='\t', index=False)

In [ ]:
or_df[['content', 'arabic_text']].to_csv('translated_data(temperature = 0.1 & top_p = 0.9).txt', sep='\t', index=False)

In [ ]:
# Download the file
files.download('translated_data(temperature = 0.1 & top_p = 0.9).txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>